In [1]:
import cv2
import numpy as np

# Load the two input images
image_bad = cv2.imread(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479.JPG")
image_good = cv2.imread(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done\DSC_2478.jpg")

# Convert the images to the Lab color space
image_bad_lab = cv2.cvtColor(image_bad, cv2.COLOR_BGR2LAB)
image_good_lab = cv2.cvtColor(image_good, cv2.COLOR_BGR2LAB)

# Split the Lab images into separate channels
l_channel_bad, a_channel_bad, b_channel_bad = cv2.split(image_bad_lab)
l_channel_good, a_channel_good, b_channel_good = cv2.split(image_good_lab)

# Perform histogram matching on the L channel
l_channel_matched = cv2.createCLAHE().apply(l_channel_bad)
l_channel_matched = cv2.equalizeHist(l_channel_matched)

# Transfer the color and contrast information from the good image to the matched L channel
a_channel_matched = a_channel_good
b_channel_matched = b_channel_good

# Merge the matched channels back into the Lab image
image_matched_lab = cv2.merge([l_channel_matched, a_channel_matched, b_channel_matched])

# Convert the matched Lab image back to BGR color space
image_matched = cv2.cvtColor(image_matched_lab, cv2.COLOR_LAB2BGR)

# Display the matched image
cv2.imshow('Matched Image', image_matched)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the VGG19 model pre-trained on ImageNet
model = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

# Define the content and style layers for style transfer
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

def preprocess_image(image_path):
    # Load and preprocess the input image
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize the image to VGG19 input size
    img = np.array(img)
    img = tf.keras.applications.vgg19.preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img

def deprocess_image(img):
    # Deprocess the image to convert it back to a valid image
    img = img.reshape((img.shape[1], img.shape[2], 3))
    img += [103.939, 116.779, 123.68]  # Undo VGG19 preprocessing
    img = np.clip(img, 0, 255).astype('uint8')
    return img

def get_feature_representations(model, content_path, style_path):
    # Preprocess the content and style images
    content_image = preprocess_image(content_path)
    style_image = preprocess_image(style_path)

    # Extract the feature representations from the model
    content_outputs = model(content_image)
    style_outputs = model(style_image)

    # Get the feature representations from the desired layers
    content_features = [content_layer[0] for content_layer in content_outputs[len(content_layers):]]
    style_features = [style_layer[0] for style_layer in style_outputs[:len(style_layers)]]

    return content_features, style_features

def compute_loss(content_features, style_features, generated_features):
    # Compute the content loss
    content_loss = tf.add_n([tf.reduce_mean(tf.square(c - g)) for c, g in zip(content_features, generated_features)])

    # Compute the style loss
    style_loss = tf.add_n([tf.reduce_mean(tf.square(s - g)) for s, g in zip(style_features, generated_features)])

    return content_loss, style_loss

def compute_grads(cfg):
    with tf.GradientTape() as tape:
        all_loss = compute_loss(**cfg)
    grads = tape.gradient(all_loss[0] + all_loss[1], cfg['generated_image'])
    return grads, all_loss

def run_style_transfer(content_path, style_path, num_iterations=1000, content_weight=1e3, style_weight=1e-2):
    # Load and preprocess the content and style images
    content_image = preprocess_image(content_path)
    style_image = preprocess_image(style_path)

    # Initialize the generated image as a copy of the content image
    generated_image = tf.Variable(content_image, dtype=tf.float32)

    # Get the feature representations of the content and style images
    content_features, style_features = get_feature_representations(model, content_path, style_path)

    # Set the optimizer and learning rate
    optimizer = tf.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)

    # Create a configuration dictionary for computing gradients and loss
    cfg = {
        'model': model,
        'content_features': content_features,
        'style_features': style_features,
        'generated_image': generated_image,
    }

    for i in range(num_iterations):
        # Compute gradients and loss
        grads, all_loss = compute_grads(cfg)
        loss = content_weight * all_loss[0] + style_weight * all_loss[1]

        # Update the generated image using the optimizer
        optimizer.apply_gradients([(grads, generated_image)])

        # Clip the generated image to valid range
        generated_image.assign(tf.clip_by_value(generated_image, -1, 1))

        if i % 100 == 0:
            print(f"Iteration: {i}, Total loss: {loss:.2f}, Content loss: {all_loss[0]:.2f}, Style loss: {all_loss[1]:.2f}")

    # Deprocess the final generated image
    final_image = deprocess_image(generated_image.numpy())

    return final_image

# Specify the paths to the content and style images
content_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479.JPG"
style_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done\DSC_2478.jpg"

# Run the style transfer algorithm
result_image = run_style_transfer(content_path, style_path, num_iterations=1000, content_weight=1e3, style_weight=1e-2)

# Save and display the result image
Image.fromarray(result_image).save('output_image.jpg')
Image.fromarray(result_image).show()


TypeError: compute_loss() got an unexpected keyword argument 'model'

In [8]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the VGG19 model pre-trained on ImageNet
model = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

# Define the content and style layers for style transfer
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

def preprocess_image(image_path):
    # Load and preprocess the input image
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize the image to VGG19 input size
    img = np.array(img)
    img = tf.keras.applications.vgg19.preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img

def deprocess_image(img):
    # Deprocess the image to convert it back to a valid image
    img = img.reshape((img.shape[1], img.shape[2], 3))
    img += [103.939, 116.779, 123.68]  # Undo VGG19 preprocessing
    img = np.clip(img, 0, 255).astype('uint8')
    return img

def get_feature_representations(model, content_path, style_path):
    # Preprocess the content and style images
    content_image = preprocess_image(content_path)
    style_image = preprocess_image(style_path)

    # Extract the feature representations from the model
    content_outputs = model(content_image)
    style_outputs = model(style_image)

    # Get the feature representations from the desired layers
    content_features = [content_layer[0] for content_layer in content_outputs[len(content_layers):]]
    style_features = [style_layer[0] for style_layer in style_outputs[:len(style_layers)]]

    return content_features, style_features

def compute_loss(content_features, style_features, generated_features):
    # Flatten the style and generated features
    style_shape = tf.shape(style_features[0])
    generated_shape = tf.shape(generated_features[0])
    style_features = [tf.reshape(style, (-1, style_shape[1], style_shape[2], style_shape[3])) for style in style_features]
    generated_features = [tf.reshape(generated, (-1, generated_shape[1], generated_shape[2], generated_shape[3])) for generated in generated_features]

    # Compute the content loss
    content_loss = tf.reduce_mean(tf.square(content_features - generated_features[-1]))

    # Compute the style loss
    style_loss = tf.add_n([tf.reduce_mean(tf.square(s - g)) for s, g in zip(style_features, generated_features[:-1])])

    return content_loss, style_loss



def compute_grads(cfg):
    with tf.GradientTape() as tape:
        all_loss = compute_loss(cfg['content_features'], cfg['style_features'], cfg['generated_image'])
    grads = tape.gradient(all_loss[0] + all_loss[1], cfg['generated_image'])
    return grads, all_loss

def run_style_transfer(content_path, style_path, num_iterations=1000, content_weight=1e3, style_weight=1e-2):
    # Load and preprocess the content and style images
    content_image = preprocess_image(content_path)
    style_image = preprocess_image(style_path)

    # Initialize the generated image as a copy of the content image
    generated_image = tf.Variable(content_image, dtype=tf.float32)

    # Get the feature representations of the content and style images
    content_features, style_features = get_feature_representations(model, content_path, style_path)

    # Set the optimizer and learning rate
    optimizer = tf.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)

    # Create a configuration dictionary for computing gradients and loss
    cfg = {
        'content_features': content_features,
        'style_features': style_features,
        'generated_image': generated_image,
    }

    for i in range(num_iterations):
        # Compute gradients and loss
        grads, all_loss = compute_grads(cfg)
        loss = content_weight * all_loss[0] + style_weight * all_loss[1]

        # Update the generated image using the optimizer
        optimizer.apply_gradients([(grads, generated_image)])

        # Clip the generated image to valid range
        generated_image.assign(tf.clip_by_value(generated_image, -1, 1))

        if i % 100 == 0:
            print(f"Iteration: {i}, Total loss: {loss:.2f}, Content loss: {all_loss[0]:.2f}, Style loss: {all_loss[1]:.2f}")

    # Deprocess the final generated image
    final_image = deprocess_image(generated_image.numpy())

    return final_image

# Specify the paths to the content and style images
content_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479.JPG"
style_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done\DSC_2478.jpg"

# Run the style transfer algorithm
result_image = run_style_transfer(content_path, style_path, num_iterations=1000, content_weight=1e3, style_weight=1e-2)

# Save and display the result image
Image.fromarray(result_image).save('output_image.jpg')
Image.fromarray(result_image).show()


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 2 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/

In [14]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the VGG19 model pre-trained on ImageNet
model = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

# Define the content and style layers for style transfer
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

def preprocess_image(image_path, target_size=(224, 224)):
    # Load and preprocess the input image
    img = Image.open(image_path)
    img = img.resize(target_size)  # Resize the image to the target size
    img = np.array(img)
    img = tf.keras.applications.vgg19.preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img

def deprocess_image(img):
    # Deprocess the image to convert it back to a valid image
    img = img.reshape((img.shape[1], img.shape[2], 3))
    img += [103.939, 116.779, 123.68]  # Undo VGG19 preprocessing
    img = np.clip(img, 0, 255).astype('uint8')
    return img

def get_feature_representations(model, content_path, style_path):
    # Preprocess the content and style images
    content_image = preprocess_image(content_path)
    style_image = preprocess_image(style_path)

    # Extract the feature representations from the model
    content_outputs = model(content_image)
    style_outputs = model(style_image)

    # Get the feature representations from the desired layers
    content_features = [content_layer[0] for content_layer in content_outputs[len(content_layers):]]
    style_features = [style_layer[0] for style_layer in style_outputs[:len(style_layers)]]

    return content_features, style_features

def compute_loss(content_features, style_features, generated_features):
    # Ensure content_features and generated_features have the same shape
    target_shape = (tf.shape(content_features)[0], tf.shape(content_features)[1], tf.shape(content_features)[2])
    target_shape = tf.shape(content_features)
    generated_features = tf.image.resize(generated_features, target_shape[1:3])

    # Compute the content loss
    content_loss = tf.reduce_mean(tf.square(content_features - generated_features))

    # Compute the style loss
    style_loss = tf.add_n([tf.reduce_mean(tf.square(s - g)) for s, g in zip(style_features, generated_features)])

    return content_loss, style_loss


def compute_grads(cfg):
    with tf.GradientTape() as tape:
        all_loss = compute_loss(cfg['content_features'], cfg['style_features'], cfg['generated_image'])
    grads = tape.gradient(all_loss[0] + all_loss[1], cfg['generated_image'])
    return grads, all_loss

def run_style_transfer(content_path, style_path, num_iterations=1000, content_weight=1e3, style_weight=1e-2):
    # Load and preprocess the content and style images
    content_image = preprocess_image(content_path)
    style_image = preprocess_image(style_path)

    # Initialize the generated image as a copy of the content image
    generated_image = tf.Variable(content_image, dtype=tf.float32)

    # Get the feature representations of the content and style images
    content_features, style_features = get_feature_representations(model, content_path, style_path)

    # Set the optimizer and learning rate
    optimizer = tf.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)

    # Create a configuration dictionary for computing gradients and loss
    cfg = {
        'content_features': content_features,
        'style_features': style_features,
        'generated_image': generated_image,
    }

    for i in range(num_iterations):
        # Compute gradients and loss
        grads, all_loss = compute_grads(cfg)
        loss = content_weight * all_loss[0] + style_weight * all_loss[1]

        # Update the generated image using the optimizer
        optimizer.apply_gradients([(grads, generated_image)])

        # Clip the generated image to valid range
        generated_image.assign(tf.clip_by_value(generated_image, -1, 1))

        if i % 100 == 0:
            print(f"Iteration: {i}, Total loss: {loss:.2f}, Content loss: {all_loss[0]:.2f}, Style loss: {all_loss[1]:.2f}")

    # Deprocess the final generated image
    final_image = deprocess_image(generated_image.numpy())

    return final_image

# Specify the paths to the content and style images
content_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479.JPG"
style_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done\DSC_2478.jpg"

# Run the style transfer algorithm
result_image = run_style_transfer(content_path, style_path, num_iterations=1000, content_weight=1e3, style_weight=1e-2)

# Save and display the result image
Image.fromarray(result_image).save('output_image.jpg')
Image.fromarray(result_image).show()


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 1 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the content image and style image
content_image = Image.open(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479.JPG")
style_image = Image.open(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479.JPG")

# Preprocess the images
preprocess = tf.keras.applications.vgg19.preprocess_input
content_array = preprocess(np.expand_dims(content_image, axis=0))
style_array = preprocess(np.expand_dims(style_image, axis=0))

# Create a VGG19 model
vgg19 = tf.keras.applications.VGG19(include_top=False, weights="imagenet")

# Get the content and style feature maps
content_features = vgg19(content_array)["block5_conv2"]
style_features = vgg19(style_array)["block5_conv2"]

# Define the target shape for the generated image
target_shape = content_features.shape

# Create a random generated image with the same shape as the target shape
generated_image = tf.Variable(tf.random.uniform(target_shape, minval=0, maxval=255, dtype=tf.float32))

# Define the content weight and style weight
content_weight = 1e3
style_weight = 1e-2

# Define the optimizer
optimizer = tf.optimizers.Adam(learning_rate=5)

@tf.function
def compute_loss(content_features, style_features, generated_features):
    # Compute the content loss
    content_loss = tf.reduce_mean(tf.square(content_features - generated_features))
    
    # Compute the style loss
    style_loss = tf.add_n([tf.reduce_mean(tf.square(s - g)) for s, g in zip(style_features, generated_features)])
    
    return content_loss, style_loss

@tf.function
def compute_grads(content_features, style_features, generated_image):
    with tf.GradientTape() as tape:
        generated_features = vgg19(generated_image)["block5_conv2"]
        loss = compute_loss(content_features, style_features, generated_features)
    grads = tape.gradient(loss[0] * content_weight + loss[1] * style_weight, generated_image)
    optimizer.apply_gradients([(grads, generated_image)])
    return loss

def run_style_transfer(content_image_path, style_image_path, num_iterations, content_weight, style_weight):
    # Load the content and style images
    content_image = Image.open(content_image_path)
    style_image = Image.open(style_image_path)
    
    # Preprocess the images
    preprocess = tf.keras.applications.vgg19.preprocess_input
    content_array = preprocess(np.expand_dims(content_image, axis=0))
    style_array = preprocess(np.expand_dims(style_image, axis=0))
    
    # Create a VGG19 model
    vgg19 = tf.keras.applications.VGG19(include_top=False, weights="imagenet")
    
    # Get the content and style feature maps
    content_features = vgg19(content_array)["block5_conv2"]
    style_features = vgg19(style_array)["block5_conv2"]
    
    # Define the target shape for the generated image
    target_shape = content_features.shape
    
    # Create a random generated image with the same shape as the target shape
    generated_image = tf.Variable(tf.random.uniform(target_shape, minval=0, maxval=255, dtype=tf.float32))
    
    # Define the optimizer
    optimizer = tf.optimizers.Adam(learning_rate=5)
    
    # Run the style transfer algorithm
    for i in range(num_iterations):
        loss = compute_grads(content_features, style_features, generated_image)
        if i % 100 == 0:
            print(f"Iteration: {i}, Content Loss: {loss[0]}, Style Loss: {loss[1]}")
    
    # Convert the generated image to the expected range
    generated_image = tf.clip_by_value(generated_image, 0, 255)
    generated_image = tf.cast(generated_image, tf.uint8)
    
    # Return the generated image
    return generated_image.numpy()

# Set the paths and other parameters
content_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479.JPG"
style_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw\DSC_2479.JPG"
num_iterations = 1000
content_weight = 1e3
style_weight = 1e-2

# Run the style transfer algorithm
result_image = run_style_transfer(content_path, style_path, num_iterations, content_weight, style_weight)

# Save and display the result image
Image.fromarray(result_image[0]).save(r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done\DSC_2478dundo.jpg")
